In [ ]:
import time
import inout

start = time.time()

# Interface to load from T0 to TN both in Master and Slave time (-->square Covariance matrices)

selSet = "WWIELKOPOLSKA_4x19_UTM"
dateT0 = "2017-01-01"
dateTN = "2017-04-01"

dateTAG = '2017Q1_N'

odir = './'

inout.print_coverages()

print("Selected data set: " + selSet)
print("  Initial date: " + dateT0 + " End date: " + dateTN)


In [ ]:
import inout 

geo_proj = inout.getProjection(selSet)
geo_transf = inout.getGeoTransform(selSet)

print(geo_transf)

In [ ]:

ncdata = inout.loadSqData(selSet, dateT0, dateTN)

load_end = time.time()
print('Elapsed time loading: ')
print(load_end - start)



In [ ]:
# plot one image from each of the two variables into a subplot for direct comparison.
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors

coh_vh = ncdata['Coherence_VH'][0][0][:][:].values.astype(np.float)
coh_vv = ncdata['Coherence_VV'][0][0][:][:].values.astype(np.float)


f, axarr = plt.subplots(1, 2, sharey=True, sharex=True)
f.set_figheight(7)
f.set_figwidth(15)
# orient the data correctly, since netcdf has a different representation of the spatial coordinates.
axarr[0].imshow(np.flipud(np.rot90(coh_vh)), cmap='jet')
axarr[0].set_title('Coherence VH')
axarr[1].imshow(np.flipud(np.rot90(coh_vv)), cmap='jet')
axarr[1].set_title('Coherence VV')


In [ ]:
import datetime 

coh_vv.shape
master_dates = ncdata.coords['master_date'].values
slave_dates = ncdata.coords['slave_date'].values

mdates = []
sdates = []
for dd in ncdata.coords['master_date'].values:
        mdates.append(datetime.datetime.fromtimestamp(int(dd)))#.strftime('%Y-%m-%d'))
for dd in ncdata.coords['slave_date'].values:
        sdates.append(datetime.datetime.fromtimestamp(int(dd)))#.strftime('%Y-%m-%d'))

print(mdates)
print(sdates)



In [ ]:
from numpy import linalg as LA

eigen_start = time.time()

# ----------------------------------------------------------------------
# ----------------------------------------------------------------------
# Compute eigendecomposition over the whole scene ----------------------
# ----------------------------------------------------------------------
# ----------------------------------------------------------------------

coh_vv = ncdata['Coherence_VV'].values.astype(np.float)
coh_vh = ncdata['Coherence_VH'].values.astype(np.float)

[mm,ss]       = coh_vv[:,:,0,0].shape
[nrows,ncols] = coh_vv[0,0].shape

print(mm)
print(ss)

eigenValues_vv = np.ndarray([nrows,ncols,mm+1],dtype=float)
eigenRatios_vv = np.ndarray([nrows,ncols,4],dtype=float)
eigenValues_vh = np.ndarray([nrows,ncols,mm+1],dtype=float)
eigenRatios_vh = np.ndarray([nrows,ncols,4],dtype=float)

for ii in range(nrows):

    if not ii % 100:
        print('Row ' + str(ii) + ' / ' + str(nrows))
    
    for jj in range(ncols):

        cc_vv  = coh_vv[:,:,ii,jj]
        cc_vh  = coh_vh[:,:,ii,jj]

        COH_VV = np.eye(mm+1,dtype=coh_vv[:,:,ii,jj].dtype)/2
        COH_VH = np.eye(mm+1,dtype=coh_vv[:,:,ii,jj].dtype)/2
        
        for dd in range(mm):
            COH_VV[dd,dd+1:] = cc_vv[dd,dd:]
            COH_VH[dd,dd+1:] = cc_vh[dd,dd:]
        
        COH_VV = COH_VV + np.transpose(COH_VV)
        COH_VH = COH_VH + np.transpose(COH_VH)
        
        w,v = LA.eig(COH_VV)
        eigenValues_vv[ii,jj,:] = np.flipud(np.sort(w))
        w,v = LA.eig(COH_VH)
        eigenValues_vh[ii,jj,:] = np.flipud(np.sort(w))
        
        eigenRatios_vv[ii,jj,0] = eigenValues_vv[ii,jj,0] / sum(eigenValues_vv[ii,jj,:])
        eigenRatios_vh[ii,jj,0] = eigenValues_vh[ii,jj,0] / sum(eigenValues_vh[ii,jj,:])
        
        eigenRatios_vv[ii,jj,1] = eigenValues_vv[ii,jj,0] / eigenValues_vv[ii,jj,1]
        eigenRatios_vh[ii,jj,1] = eigenValues_vh[ii,jj,0] / eigenValues_vh[ii,jj,1]
        
        eigenRatios_vv[ii,jj,2] = eigenValues_vv[ii,jj,1] / eigenValues_vv[ii,jj,2]
        eigenRatios_vh[ii,jj,2] = eigenValues_vh[ii,jj,1] / eigenValues_vh[ii,jj,2]
        
        eigenRatios_vv[ii,jj,3] = (eigenValues_vv[ii,jj,0]-eigenValues_vv[ii,jj,1]) / (eigenValues_vv[ii,jj,0]+eigenValues_vv[ii,jj,1])
        eigenRatios_vh[ii,jj,3] = (eigenValues_vh[ii,jj,0]-eigenValues_vh[ii,jj,1]) / (eigenValues_vh[ii,jj,0]+eigenValues_vh[ii,jj,1])

# ----------------------------------------------------------------------
# ----------------------------------------------------------------------

eigen_end = time.time()
print('Elapsed time loading: ')
print(eigen_end - eigen_start)


In [ ]:
# plot one image from each of the two variables into a subplot for direct comparison.
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors

#eigenValues[:,:,0]

f, axarr = plt.subplots(1, 5, sharey=True, sharex=True)
f.set_figheight(7)
f.set_figwidth(15)
# orient the data correctly, since netcdf has a different representation of the spatial coordinates.
axarr[0].imshow(np.flipud(np.rot90(eigenValues_vv[:,:,0])), cmap='jet',vmin=0.0, vmax=5.)
axarr[0].set_title('Eigenvalue 1 VV')
axarr[1].imshow(np.flipud(np.rot90(eigenValues_vv[:,:,1])), cmap='jet',vmin=0.0, vmax=4.)
axarr[1].set_title('Eigenvalue 2 VV')
axarr[2].imshow(np.flipud(np.rot90(eigenValues_vv[:,:,2])), cmap='jet',vmin=0.0, vmax=3.)
axarr[2].set_title('Eigenvalue 3 VV')
axarr[3].imshow(np.flipud(np.rot90(eigenValues_vv[:,:,3])), cmap='jet',vmin=0.0, vmax=2.)
axarr[3].set_title('Eigenvalue 4 VV')
axarr[4].imshow(np.flipud(np.rot90(eigenValues_vv[:,:,4])), cmap='jet',vmin=0.0, vmax=1.)
axarr[4].set_title('Eigenvalue 5 VV')

In [ ]:
# plot one image from each of the two variables into a subplot for direct comparison.
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors

#eigenValues[:,:,0]

f, axarr = plt.subplots(1, 5, sharey=True, sharex=True)
f.set_figheight(7)
f.set_figwidth(15)
# orient the data correctly, since netcdf has a different representation of the spatial coordinates.
axarr[0].imshow(np.flipud(np.rot90(eigenValues_vh[:,:,0])), cmap='jet',vmin=0.0, vmax=3.)
axarr[0].set_title('Eigenvalue 1 VH')
axarr[1].imshow(np.flipud(np.rot90(eigenValues_vh[:,:,1])), cmap='jet',vmin=0.0, vmax=3.)
axarr[1].set_title('Eigenvalue 2 VH')
axarr[2].imshow(np.flipud(np.rot90(eigenValues_vh[:,:,2])), cmap='jet',vmin=0.0, vmax=3.)
axarr[2].set_title('Eigenvalue 3 VH')
axarr[3].imshow(np.flipud(np.rot90(eigenValues_vh[:,:,3])), cmap='jet',vmin=0.0, vmax=3.)
axarr[3].set_title('Eigenvalue 4 VH')
axarr[4].imshow(np.flipud(np.rot90(eigenValues_vh[:,:,4])), cmap='jet',vmin=0.0, vmax=3.)
axarr[4].set_title('Eigenvalue 5 VH')

In [ ]:
# plot one image from each of the two variables into a subplot for direct comparison.
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors

f, axarr = plt.subplots(1, 4, sharey=True, sharex=True)
f.set_figheight(7)
f.set_figwidth(15)
# orient the data correctly, since netcdf has a different representation of the spatial coordinates.
axarr[0].imshow(np.flipud(np.rot90(eigenRatios_vv[:,:,0])), cmap='jet',vmin=0.0, vmax=1.)
axarr[0].set_title('lambda_1/sum(lambda)')
axarr[1].imshow(np.flipud(np.rot90(eigenRatios_vv[:,:,1])), cmap='jet',vmin=0.0, vmax=3.)
axarr[1].set_title('lambda_1/lambda_2')
axarr[2].imshow(np.flipud(np.rot90(eigenRatios_vv[:,:,2])), cmap='jet',vmin=0.0, vmax=3.)
axarr[2].set_title('lambda_2/lambda_3')
axarr[3].imshow(np.flipud(np.rot90(eigenRatios_vv[:,:,3])), cmap='jet',vmin=0.0, vmax=1.)
axarr[3].set_title('(lambda_1-lambda_2)/(lambda_1+lambda_2)')

In [ ]:
# plot one image from each of the two variables into a subplot for direct comparison.
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors

f, axarr = plt.subplots(1, 4, sharey=True, sharex=True)
f.set_figheight(7)
f.set_figwidth(15)
# orient the data correctly, since netcdf has a different representation of the spatial coordinates.
axarr[0].imshow(np.flipud(np.rot90(eigenRatios_vh[:,:,0])), cmap='jet',vmin=0.0, vmax=1.)
axarr[0].set_title('lambda_1/sum(lambda)')
axarr[1].imshow(np.flipud(np.rot90(eigenRatios_vh[:,:,1])), cmap='jet',vmin=0.0, vmax=3.)
axarr[1].set_title('lambda_1/lambda_2')
axarr[2].imshow(np.flipud(np.rot90(eigenRatios_vh[:,:,2])), cmap='jet',vmin=0.0, vmax=3.)
axarr[2].set_title('lambda_2/lambda_3')
axarr[3].imshow(np.flipud(np.rot90(eigenRatios_vh[:,:,3])), cmap='jet',vmin=0.0, vmax=1.)
axarr[3].set_title('(lambda_1-lambda_2)/(lambda_1+lambda_2)')

In [ ]:
import os
from geo_utils import write_geotiff



eigenValuesVV_file = os.path.join(odir,selSet+'_'+ dateTAG +'_eigValuesVV.tiff')
write_geotiff(eigenValuesVV_file, np.flipud(np.rot90(eigenValues_vv)), geo_transf, geo_proj)
#write_geotiff(eigenValuesVV_file, eigenValues_vv, geo_transf, geo_proj)

eigenValuesVH_file = os.path.join(odir,selSet+'_'+ dateTAG +'_eigValuesVH.tiff')
write_geotiff(eigenValuesVH_file, np.flipud(np.rot90(eigenValues_vh)), geo_transf, geo_proj)
#write_geotiff(eigenValuesVH_file, eigenValues_vh, geo_transf, geo_proj)

ratioVV_file = os.path.join(odir,selSet+'_' + dateTAG +'_eigRatiosVV.tiff')
write_geotiff(ratioVV_file, np.flipud(np.rot90(eigenRatios_vv)), geo_transf, geo_proj)
#write_geotiff(ratioVV_file, eigenRatios_vv, geo_transf, geo_proj)

ratioVH_file = os.path.join(odir,selSet+'_'+ dateTAG +'_eigRatiosVH.tiff')
write_geotiff(ratioVH_file, np.flipud(np.rot90(eigenRatios_vh)), geo_transf, geo_proj)
#write_geotiff(ratioVH_file, eigenRatios_vh, geo_transf, geo_proj)
